# 10 List Generator

In section 09, we have seen list comprehension. List comprehensions are useful and can help you write elegant code that’s easy to read and debug, but they’re not the right choice for all circumstances.

For example, when you handle a big list, list comprehension may consume all your memory. Because **A list comprehension in Python works by loading the entire output list into memory**.

To solve this problem, we need to use a **generator** instead of a list comprehension. **A generator doesn’t create a single, large data structure in memory, but instead returns an iterable**.

## 10.1 Create a list generator

There are many ways to create a generator，the simplest way is to use below expression

```python
(expression for member in iterable)
```
- expression: is the member itself, a call to a method, or any other valid expression that returns a value.
- member: is the object or value in the list or iterable. In the example above, the member value is i.
- iterable: is a list, set, sequence, generator, or any other object that can return its elements one at a time.

Check the below example, and notice the difference between their types
- list_comprehension is a list
- list_generator is a generator(iterator)

In [4]:
from collections.abc import generator

list_comprehension = [x * x for x in range(10)]
print(type(list_comprehension))

list_generator = (x * x for x in range(10))
print(type(list_generator))

<class 'list'>
<class 'generator'>


Unlike list, generator does not have a **toString method** to print it's content. That's because it does not know the content, it only memorises the function on how to calculate the next item, so it does not know the entire list. To get the next element, we need to call **next()**

In [5]:
print(list_generator)

<generator object <genexpr> at 0x7f8d20161c80>


In [6]:
print(next(list_generator))

0


Notice the index of generator starts from the first element of the list, and increments 1 after we call next. So if we call next() again, it will return the 2nd element

In [7]:
print(next(list_generator))

1


If the next element is out of the list range, you will receive **StopIteration error**

In [9]:
try:
    while True:
        print(next(list_generator))
except StopIteration as e:
    print(e)

To avoid this, you can use the **for loop to handle the iteration without receiving error**, and **you don't need to call the next() method anymore**.
Note **generator is unidirectional (just like any normal iterator), you can not go back. If you want to start from the beginning again, you need to create a new generator.

In [10]:
list_generator = (x * x for x in range(10))
for item in list_generator:
    print(item)

0
1
4
9
16
25
36
49
64
81


## 10.2 Generators function in Python

There is a lot of work in building an iterator in Python. **We have to implement a class with __iter__() and __next__() method, keep track of internal states, and raise StopIteration when there are no values to be returned**.

This is both lengthy and counterintuitive. Generator comes to the rescue in such situations.

Python generators are a simple way of creating iterators. All the work we mentioned above are automatically handled by generators in Python.

Simply speaking, **a generator is a function that returns an object (iterator) which we can iterate over (one value at a time).**

### 10.2.1 Differences between Generator function and Normal function
Here is how a generator function differs from a normal function.
- Generator function contains **one or more yield statements**.
- When called, **it returns an object (iterator) but does not start execution immediately.**
- Methods like __iter__() and __next__() are implemented automatically. So we can **iterate through the items using next().**
- Once the function yields(when the execution encounter the yield statement), **the function is paused and the control is transferred to the caller**.
- Local variables and their states are remembered between successive calls.
- Finally, when the function terminates, **StopIteration is raised automatically on further calls. If it has return statement, the return value is in StopIteration.value

### 10.2.2 Illustrate a simple generator function


In [2]:
def simple_generator_fun():
    data = 10
    print("This is the first pause")
    data -= 1
    yield data
    print("This is the second pause")
    data -= 1
    yield data
    print("This is the third pause")
    data -= 1
    yield data
    return "Finish simple_generator_fun iteration"


gf = simple_generator_fun()
print(type(gf))

<class 'generator'>


In [7]:
# start the iteration and get the first item
i1 = next(gf)
print(i1)

This is the first pause
9


In [8]:
# continue the iteration and get the second item
i2 = next(gf)
print(i2)

This is the second pause
8


In [9]:
# continue the iteration and get the third item
i3 = next(gf)
print(i3)

This is the third pause
7


**You can notice that the state of local variable data is conserved during the iteration.** Unlike normal functions, the local variables are not destroyed when the function yields.

In [5]:
# if we continue the iteration, we will get an error, because there is no more yield left to iterate
i4 = next(gf)
print(i4)

StopIteration: 

Furthermore, the generator object can be iterated only once, and you can't iterate backwards

To avoid the StopIteration error, you can use the for loop to handle it automatically. This is because a for loop takes an iterator and iterates over it using next() function. It automatically ends when StopIteration is raised.

In [10]:
new_gf = simple_generator_fun()

for item in new_gf:
    print(item)

This is the first pause
9
This is the second pause
8
This is the third pause
7


### 10.2.2 Python Generators with a Loop

Normally, generator functions are implemented with a loop having a suitable terminating condition.

Let's take an example of a generator that reverses a string

In [11]:
def rev_str(my_str):
    length = len(my_str)
    # range start index is (length-1), end index is -1(as it's not inclusive, so end index is 0), and step is -1
    for i in range(length - 1, -1, -1):
        yield my_str[i]


# For loop to reverse the string
for char in rev_str("hello"):
    print(char)

o
l
l
e
h


### 10.2.3 Python Generators return value

Normally, we don't use the return value of a generator, but if you want to use it. You can get it through StopIteration.value

In [3]:
sgf = simple_generator_fun()
while True:
    try:
        next(sgf)
    except StopIteration as e:
        print(f"the return value of simple_generator_fun is: {e.value}")
        break

This is the first pause
This is the second pause
This is the third pause
the return value of simple_generator_fun is: Finish simple_generator_fun iteration


## 10.3 When to use python generator

- Simplify your code
- Memory Efficient : A normal function to return a sequence will create the entire sequence in memory before returning the result. This is an overkill, if the
                     number of items in the sequence is very large. Generator implementation of such sequences is memory friendly and is preferred since it only produces one item at a time.
- Represent Infinite Stream: Generators are excellent mediums to represent an infinite stream of data. Infinite streams cannot be stored in memory, and since
                             generators produce only one item at a time, they can represent an infinite stream of data

- Pipelining Generators


### 10.3.1 Simplify your code

Generators can be implemented in a clear and concise way as **compared to iterator class**. Following is an example to implement a sequence of power of 2 using an iterator class.

In [ ]:
class PowerTwo:
    def __init__(self, max):
        self.current = 0
        self.max = max

    def __iter__(self):
        return self

    def __next__(self):
        if self.current > self.max:
            raise StopIteration
        result = 2 ** self.current
        self.current += 1
        return result


The above program was lengthy and confusing. Now, let's do the same using a generator function.

In [6]:
def power_two_generator(max: int):
    current = 0
    while current <= max:
        yield 2 ** current
        current += 1

In [7]:
p2g = power_two_generator(5)
for item in p2g:
    print(item)

1
2
4
8
16
32


### 10.3.2 Represent Infinite Stream
The following generator function can generate all the even numbers(at least in theory).

In [8]:
def all_even():
    n = 0
    while True:
        yield n
        n += 2

### 10.3.3 Pipelining Generators

Multiple generators can be used to pipeline a series of operations. This is best illustrated using an example.

Suppose we have a generator that produces the numbers in the Fibonacci series. And we have another generator for squaring numbers.

If we want to find out the sum of squares of numbers in the Fibonacci series, we can do it in the following way by pipelining the output of generator functions together.


Note the expression `a,b=b,a+b` is similar

```
t = (b, a + b) # t is a tuple
a = t[0]
b = t[1]
```

In [9]:
def fib(max):
    a, b = 0, 1
    for _ in range(max):
        yield b
        a, b = b, a + b
    return 'done'

In [11]:
fib_g = fib(5)
for i in fib_g:
    print(i)

1
1
2
3
5


Below function square takes a generator (or any iterable object)

In [14]:
def square(nums):
    for num in nums:
        yield num*num

In [15]:
print(sum(square(fib(5))))

40


## 10.4 Exercise

          1
         / \
        1   1
       / \ / \
      1   2   1
     / \ / \ / \
    1   3   3   1
   / \ / \ / \ / \
  1   4   6   4   1
 / \ / \ / \ / \ / \
1   5   10  10  5   1

Above is an example of yanghui triangle, which can resolve equations

Consider each level is a list, write a generator which can return a list of numbers that represent the level base on the given level number

In [27]:
def generate_triangle(max):
    n=0
    current=[]
    while n<=max:
        res=[]
        for i in range(0,n+1):
            if i==0 or i==n:
                res.append(1)
            else:
                res.append(current[i-1]+current[i])
        yield res
        current=res
        n+=1


In [28]:
tri=generate_triangle(5)
for item in tri:
    print(item)

[1]
[1, 1]
[1, 2, 1]
[1, 3, 3, 1]
[1, 4, 6, 4, 1]
[1, 5, 10, 10, 5, 1]
